<h1>CS4619: Artificial Intelligence II</h1>
<h1>Sequence Models</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

<h1>Acknowledgements</h1>
<ul>
    <li>The image captioning example comes from a talk by Jeff Dean of Google at 
        <i>AI Frontiers: Trends and Developments in Deep Learning Research</i>:
        <a href="https://www.slideshare.net/AIFrontiers/jeff-dean-trends-and-developments-in-deep-learning-research">https://www.slideshare.net/AIFrontiers/jeff-dean-trends-and-developments-in-deep-learning-research</a>
    </li>
</ul>

<h1>Sequence Data Applications</h1>
<ul>
    <li>Let's start by reviewing some applications that use sequence data. We will group them, depending on
        whether they are sequence-to-vector, vector-to-sequence or sequence-to-sequence.
    </li>
    <li>Sequence-to-vector (also called many-to-one), e.g.:
        <ul>
            <li>Timeseries forecasting, taking in sequences of numbers (historical rainfall data, stock prices, sales) and predicting the next number (tomorrow's rainfall, stock price, sales)
            </li>
            <li>Sentiment classification, taking in text, outputting a class (e.g. positive, negative) or a score</li>
            <li>Activity recognition from video, taking in a video (sequence of images), outputting a class (e.g. running, walking, crawling)</li>
        </ul>
    </li>
    <li>Vector-to-sequence (also called one-to-many), e.g.:
        <ul>
            <li>Image captioning, taking in an image, outputting a descriptive phrase (sequence of words)</li>
            <li>Music generation (perhaps), taking in a first note, outputting subsequent notes</li>
        </ul>
    </li>
    <li>Sequence-to-sequence (also called many-to-many), e.g.:
        <ul>
            <li>Speech-to-text and text-to-speech: 
                <span style="float: right"><img style="display: inline;" src="images/soundwave.png" />
                    Hey, Google. When does Woodies open?</span>
            </li>
            <li style="clear: both;">DNA analysis, taking in a DNA sequence, outputting a corresponding sequence but highlighting a subsequence
                within the input that codes a certain protein, for example AGCCCCTGTGAGGAACTAG $\rightarrow$ 0011111111111111100
            </li>
            <li>Named entity recognition, taking in text (sequence of words), outputting a corresponding sequence but highlighting subsequences that are the names of people, organizations, etc. "Elon Musk works for SpaceX and Tesla Inc." $\rightarrow$ 11001011
                <ul>
                    <li>Named entity recognition has all sorts of uses. It might be used by search engines when indexing documents, or by customer support departments when routing your queries and complaints &mdash; to give just two examples.
                    </li>
                    <li>Rather than outputting bit strings (0s and 1s), it can be more useful if the system outputs symbols that show where a name starts and where it ends.
                    </li>
                </ul>
            </li>
            <li>Machine translation, taking in text in one language, outputting text in another language "La plume de ma tante est sur la table" $\rightarrow$ "My aunt's pen is on the table"
            </li>
        </ul>
    </li>
    <li>In previous lectures, we have seen an example of a sequence-to-vector application, namely sentiment classification.
    </li>
    <li>So, in this lecture, let's give examples of vector-to-sequence (image captioning) and
        sequence-to-sequence (machine translation and question-answering).
    </li>
</ul>

<h1>Image Captioning</h1>
<ul>
    <li>Image captioning is a vector-to-sequence application.
        <figure>
            <img src="images/image_captioning.jpg" />
        </figure>
    </li>
    <li>We need a dataset of images plus human captions. 
    </li>
    <li>This diagram gives a schematic idea of the kind of neural network architecture that we need:
        <figure>
            <img src="images/captioning_architecture.png" />
        </figure>
    </li>
    <li>We can think of this as an encoder-decoder architecture.</li>
    <li>The convolutional network on the left of the diagram is the encoder: it produces a vector that captures the content of the image.
        <ul>
            <li>It is like the convolutional networks we studied before: convolutional layers, max-pooling layers,
                and so on.
            </li>
            <li>However, we use the convolutional base of the networks we saw before: we do not include the 
                flattening layer and the dense layers. We are not trying to classify images (e.g. cat or dog).
                We're trying to encode the image in a vector of features that captures the content of the image.
                This vector is passed into the decoder part of the network.
            </li>
        </ul>
    </li>
    <li>The RNN on the right of the diagram is the decoder: it takes in the vector that encodes the images and then,
        in each timestep, outputs each word of the caption.
        <ul>
            <li>The decoder is like the language model from previously. But instead of feeding in a vector of all zeros,
        we feed in the output of the encoder.
            </li>
        </ul>
    </li>
</ul>

<h2>Training the image captioning system</h2>
<ul>
    <li>The basic idea:
        <ul>
            <li>It's easier to think about a single training example, rather than a batch.</li>
            <li>The image is fed into the convolutional network, producing a vector as output of that network.</li>
            <li>Each word of the actual caption is fed into the RNN in turn, starting with the start-of-sentence token and ending with the end-of-sentence token. At each timestep, the RNN must predict the next word.
                <ul>
                    <li>If the RNN predicts that word, then there are no weight updates.</li>
                    <li>If it predicts a different word, then the weights in the RNN and the ConvNet will get updated.</li>
                </ul>
            </li>
        </ul>
    </li>
    <li>There are all sorts of complications, including:
        <ul>
            <li>As we discussed in a previous lecture, softmax computations for a word-level model will be
                expensive, so you may want to use softmax sampling instead.
            </li>
            <li>How do we handle training sentences of various lengths? We can pad but we can't crop. Why not?
                Common
                instead is for batches to contain sentences of the same length.
            </li>
        </ul>
    </li>
</ul>

<h2>Predictions using the image captioning system</h2>
<ul>
    <li>An image is fed into the convolutional network, producing a vector as the output of that network.</li>
    <li>The start-of-sentence token is fed into the RNN.</li>
    <li>Again, at each timestep, the RNN predicts the next word.
        <ul>
            <li>To be more exact, for each word in the vocabulary, it predicts how probable that word is.</li>
            <li>And it can choose to output the one with the highest probability (but see below).</li>
        </ul>
        At the next timestep, the RNN receives (an encoding of) whichever word it chose to output.
    </li>
    <li>Again there are complications. The main one is that this is a <b>greedy algorithm</b>. It outputs
        the most probable word. But choosing the most probable word at each step may not lead to the
        best caption. A solution is to use <b>beam search</b>. We exemplify a little when we discuss
        machine translation, below.
    </li>
</ul>

<h2>Examples of image captioning systems</h2>
<ul>
    <li>Google have learned a model for automatically captioning images.</li>
    <li>The neural network combines a convolutional network to find features in the image
        (using the Inception V3 model) with a recurrent neural network using LSTMs:
        <figure>
            <img src="images/captioning.png" />
            <figcaption>
                Google's image captioning system<br /> See
                <a href="https://research.googleblog.com/2016/09/show-and-tell-image-captioning-open.html">https://research.googleblog.com/2016/09/show-and-tell-image-captioning-open.html</a><br />
                Image comes from Vinyals et al.: <i>Show and Tell: Lessons learned from the 
                2015 MSCOCO Image Captioning Challenge</i>, CoRR, abs/1609.06647, 2016 
                (<a href="https://arxiv.org/pdf/1609.06647.pdf">https://arxiv.org/pdf/1609.06647.pdf</a>)
            </figcaption>
        </figure>
    </li>
    <li>Take a look at this <a href="https://twitter.com/JanelleCShane/status/969239712190746624?s=19">Twitter thread about sheep</a>.
    </li>
</ul>

<h1>Sequence-to-Sequence</h1>
<ul>
    <li>In some sequence-to-sequence applications, the input sequence and the output sequence have the same lengths.
        <ul>
            <li>This is usually because the first output corresponds to the first input; the second output corresponds
                to the second input; and so on.
            </li>
            <li>E.g. the DNA analysis and named entity recognition examples above</li>
            <li>Here's a diagram of such a network unrolled through time:
                <img src="images/seq_2_seq_same.png" />
            </li>
        </ul>
    </li>
    <li>In other applications, the input and output lengths may differ. 
        <ul>
            <li>E.g. the spech-to-text, text-to-speech and machine translation examples above.</li>
            <li>An obvious solution is to force them to be the same length by inputting or outputting 
                special dummy values.
            </li>
            <li>But more common is a different neural network architecture, an encoder-decoder network, consisting of two parts
                <ul>
                    <li>The first part, the encoder, processes the input sequence, but supresses its outputs (<code>return_sequences=False</code>).
                    </li>
                    <li>The second part, the decoder, takes the final output vector from the encoder and then
                        decodes, producing each part of the output sequence.
                    </li>
                </ul> 
                Here it is unrolled through time:
                <img src="images/seq_2_seq_different.png" />
                (Tne diagram shows the input and output as the same length, but they can be different. The last
                input and output would be EOS, the end-of-sentence token.)
            </li>
        </ul>
    </li>
</ul>

<h1>Machine Translation</h1>
<ul>
    <li>For concreteness, let's suppose we are translating from French to English.</li>
    <li>Datasets of parallel text are often available, e.g. proceedings of the Canadian parliament
        are published in both English and French. These give our training set.
        <ul>
            <li>The words may be assigned ids and these may be one-hot encoded.</li>
            <li>Or we might include a word embedding layer (not shown in our diagrams).</li>
        </ul>
    </li>
    <li>We want a sequence-to-sequence model but the one where the input and output sequences can be of different
        lengths, i.e. an encoder folllowed by a decoder.
        <ul>
            <li>In effect, the output of the encoder is a vector that represents the input sentence (French).
            <li>Instead of feeding the decoder 
                a vector of all zeros, we feed it the output of the encoder.
            </li>
            <li>As with image captioning, it's surprising that this works quite well.</li>
        </ul>
    </li>
    <li>Training: Similar to the image captioning system. We want it to predict the next word of the English 
        sentence.
    </li>
    <li>Prediction: Similar to the image captioning system. The word it chooses to output is fed in as an
        input in the next timestep.
    </li>
    <li>All the complications that we mentioned for image captioning also apply here. But let's investigate
        two in more detail: beam search and bidirectionality.
    </li>
</ul>

<h2>Beam search</h2>
<ul>
    <li>For translating a French sentence into English (i.e. prediction rather than training), we assumed 
        a <b>greedy algorithm</b>: it outputs
        the most probable word at each timestep. But choosing the most probable word at each step may not lead to the
        best overall translation. 
    </li>
    <li>For example, suppose the training set contains many occurrences of examples like "Je marche partout"/"I walk everywhere", "Je marche tous le temps"/"I walk everywhere", "Je marche chaque jour"/"I walk every day" but it contains few occurrences of sentences such as "Je marche dans la rue maintenant"/"I am walking down the street now". And suppose we are translating "Je marche vers toi maintenant" into English. A good translation is "I am walking towards you now". A poorer tranlsation is "I walk towards you now". But working greedily may not produce this translation. We can imagine that, given the examples it has been trained on, after outputting "Je", the network greedily chooses "walk" rather than "am"."
    </li>
    <li>A solution is to use <b>beam search</b>. Beam search uses a parameter, often designated $k$, called the <b>beam width</b>. In high-level terms, what it does is to keep track of the $k$ most promising output sentences. $k$ will usually be 10 and is unlikely to be as high as 100.
    </li>
    <li>A bit more detail, supposing $k=3$ and the vocabulary has 10,000 words.:
        <ul>
            <li>Instead of choosing the most probable first word from the 10,000 candidates, the network would choose the three most probable first words. Let's say they are "I", "me" and "myself".</li>
            <li>It would make three copies of the network, one for each of the three possible first words.</li>
            <li>When choosing the second word, one network considers which of the 10,000 words are most probable given that "I" is the first word. The second network considers which of them are most probable given that "me" is the first word. And the third network has "myself" as the first word. So we are looking at $3 \times 10,0000$ candidates, and we choose the three most probable of these two-word phrases. (N.B. Not three per model, but just the three best overall.) Let's say these are "I walk", "I am" and "me walk". (Note how, in this example, "myself" is no longer a candidate for the first word.)
            </li>
            <li>We instantiate our three networks with these three two-word phrases to choose the third word, so again 30,000 possibilities. Maybe the winners are "I walk towards", "I walk at" and "I am walking".
            </li>
            <li>And this continues. Eventually, we end up with three completed sentences (ones which produced the end-of-sentence token) and we select the most probable of these as the translation of the French into English.
            </li>
        </ul>
    </li>
    <li>As usual, there are some complications whose details we will ignore: one of these is that
        shorter sentences will be preferred (can you see why?) and 
        something called length normalization is the solution to this.
    </li>
</ul>

<h2>Bidirectional RNNs</h2>
<ul>
    <li>In the RNNs that we have studied so far, the output of the network at time $t$, $\hat{\v{y}}$, depends on
        the previous inputs and the current input, $\v{x}_{(0)},\ldots,\v{x}_{(t)}$.
        <ul>
            <li>e.g. the previous words of the sentence</li>
            <li>e.g. historical rainfall or sales figures</li>
        </ul>
    </li>
    <li>In a <b>bidirectional RNN</b>, $\hat{\v{y}}$ can depend on $\v{x}_{(0)},\ldots,\v{x}_{(t-1)}$ but also
        on later inputs $\v{x}_{(t+1)},\dots,\v{x}_{(\mathit{maxlen})}$
        <ul>
            <li>e.g. later words in the sentence</li>
        </ul>
    </li>
    <li>Why might this be useful?
        <ul>
            <li>E.g. in named entity recognition, suppose the input so far has been "He said:", and so you have ouput 
                00 (i.e. these are not names). Then the next word is "Bill". Should you output 0 (not a name) or 1
                (name)?
            </li>
            <li>If the word after "Bill" is "me" ("He said: Bill me!"), then you should output 0 .</li>
            <li>If the word after "Bill" is (e.g.) "is" (e.g. "He said: Bill is a fool"), then you should output 1.</li>
            <li>E.g. in machine translation from English to French, suppose the input the current word is "the".
                Should the output be "le", "la" or "l'"?
            </li>
            <li>It all depends on the next word ("the jacket" will require "la", "the sea" will require "le",
                "the man" will require "l'").
            </li>
            <li>These simple examples can probably be handled by beam search. But, in some cases, decisions cannot be made until something from much later is seen; for these examples, beam search may not work. Here's an English to French example. When the input is "Which of these is your jacket?", then "which" translates to "laquelle" because "jacket" in French ("veste") is feminine: "Laquelle d'entre elles est votre veste". But if the input is "Which of these is your hat?", then "which" tranlsates as "lequel" because "hat" ("chapeau") is masculine: "Lequel d'entre eux est votre chapeau?". For beam search to work here, a translation using "laquelle" and a translation using "lequel" would need to remain in the top $k$ all the way until the end of the sentence. This may happen, but it may not. By contrast, bidirectional search allows early decisions to use later information, and vice versa.
            </li>
        </ul>
    </li>
    <li>What does a Bidirectional RNN look like?
        <ul>
            <li>It has a recurrent layer that reads the words from left-to-right.</li>
            <li>It has a recurrent layer that reads the words from right-to-left.</li>
            <li>Outputs are combined, e.g. by concatenation, to feed into the next layer.</li>
        </ul>
        <img src="images/bidirectional.png" />
        What this means is that the final outputs are not ready for the next layer to 
        process until the whole input sequence has been read in. But the next layer in each of its timesteps 
        has information 'from the past'
        and 'from the future'.
    </li>
    <li>So, for named entity recognition, you would use a bidirectional RNN; for machine tranlstaion, you would use
        a bidirectional RNN as your encoder.
    </li>
</ul>

<ul>
    <li>If you want to use these, then Keras has a class for creating them: <code>Bidirectional</code>.
    </li>
</ul>

<h2>Attention mechanisms</h2>
<ul>
    <li>The encoder-decoder aproach to Machine Translation works quite well on short sentences, but not so well on
        longer ones (e.g. 30 words or more).
    </li>
    <li>The use of an <b>attention model</b> is an innovation that is having a profound impact.</li>
    <li>To avoid introducing lots of notation, we will study them in a hand-waving way!
        <ul>
            <li>You still have your encoder-decoder network, with a bidirectional encoder.</li>
            <li>As well as feeding the last encoder output into the decoder, we feed in all the outputs.
            </li>
            <li>The attention mechanism is, e.g., a weighted sum of these encoder outputs.</li>
            <li>These weights determine which encoder output (and hence, in some sense, which
                word of the French input) it will focus on at this step.
            </li>
            <li>How are these weights learned? The attention mechanism has a small neural network
                called an attention layer, which learns them.
            </li>
        </ul>
    </li>
    <li>Attention mechanisms have been added to other systems too, such as image captioning systems.</li>
    <li>There are even neural networks (such as the Tranformer) which use attention layers in place of 
        recurrent layers and convolutional layers. (There is a famous paper entitled "Attention is all you need".)
    </li>
    <li>Keras has implementations of several kinds of attention layer, if you're interested.</li>
</ul>

<h2>Examples of Machine Translation systems</h2>
<figure>
    <img src="images/translation.png" />
    <figcaption>
        Google's Machine Translation system<br /> See
        <a href="https://research.googleblog.com/2016/09/a-neural-network-for-machine.html">https://research.googleblog.com/2016/09/a-neural-network-for-machine.html</a><br />
        Image comes from Wu et al.: <i>Google's Neural Machine Translation System: Bridging the Gap between
        Human and Machine Translation</i>, CoRR, abs/1609.08144, 2016
        (<a href="https://arxiv.org/pdf/1609.08144.pdf">https://arxiv.org/pdf/1609.08144.pdf</a>)
    </figcaption>
</figure>

<h1>Question-Answering</h1>
<ul>
    <li>AI has long had the goal of producing a question-answering system, especially one that
        can hold extended conversations &mdash; it's central to the Turing Test, for example.
    </li>
    <li>Traditional chatbots can just about function in very narrow domains but they fail as soon
        as the conversation becomes more general.
    </li>
    <li>We are now seeing neural networks for doing this &mdash; quite similar to the ones for
        Machine Translation: an encoder and a decoder, trained on, e.g., social media conversations.
    </li>
    <li>E.g. Google's Meena system:
        <a href="https://ai.googleblog.com/2020/01/towards-conversational-agent-that-can.html">https://ai.googleblog.com/2020/01/towards-conversational-agent-that-can.html</a>
        <a href="https://arxiv.org/pdf/2001.09977.pdf">https://arxiv.org/pdf/2001.09977.pdf</a>
    </li>
    <li>OpenAI's Codex, mentioned in the previous lecture, is similar in some ways: the questions are in
        English; but the answers are code.
    </li>
</ul>